**Data Manipulation**

The code hidden below walks through my data manipulations. An overview of the steps follow.

1. Load data and packages
2. Create a stocks df
    
    a. Import each company data
    
    b. Create a return column equal to the adjusted close divded by previous adjusted close, and subtract one
    
    c. Create lagged columns of the data (for future use)
    
    d. Create step forward columns (for future use)
    
    e. Correct the date column formatting
    
    f. Save a list of trading days

3. Create a tweets df
    
    a. Remove tweets that seem to be written by bots
    
    b. Change post_date to date time
    
    c. Join company names in
    
    d. Make time zone correction to tweet times (i.e. UTC to EST)
    
    e. Adjust post date to period which it applies (a tweet made after trading hours is moved up to the next trading day)
    
    f. Clean up columns names
    
    g. Make final adjustment to tweet date (if not in trading days move to the next closest trading day)
    
4. Create final df

    a. Join tweet and stock price dfs
    
    b. Use VADER to create composite, negative, neutral, and positive scores per tweet
    
    c. Save final df to feather format

In [ ]:
# import pandas as pd
# import numpy as np
# from datetime import *
# import pandas_market_calendars as mcal
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# import feather


# # In[1]:

# stocks = pd.DataFrame(columns = ['Company', 'Date',
#                                   'lag1return',
#                                   'lag2return',
#                                   'lag3return',
#                                   'lag4return',
#                                   'lag5return',
#                                   'lag6return',
#                                   'lag7return',
#                                   'step1return',
#                                   'step2return',
#                                   'step3return',
#                                   'step4return',
#                                   'step5return',
#                                   'step6return',
#                                   'step7return'
#                                   ])

# companies = ['aapl', 'amzn', 'goog', 'googl', 'msft', 'tsla']

# for co in companies:
    
#     csv = co.upper() + '.csv'
#     data = pd.read_csv(csv)
    
#     data['Company'] = co
#     data = data.drop(['High', 'Low', 'Open', 'Close', 'Volume'], axis = 1)
#     data['return'] = data['Adj Close'].div(data['Adj Close'].shift(1)) - 1
    
#     for i in range(1, 8):
        
#         lag = str(i)
        
#         lr = 'lag' + lag + 'return'
                
#         temp = data.shift(periods = i, axis = 0, fill_value = 0)

#         data[lr] = temp['return']
    
#     for i in range(1, 8):
        
#         step = str(i)
        
#         sr = 'step' + step + 'return'
        
#         temp = data.shift(periods = -i, axis = 0, fill_value = 0)
        
#         data[sr] = temp['return']
    
#     stocks = stocks.append(data)

# stocks = stocks[['Company', 'Date', 'return', 'lag1return',
#         'lag2return', 'lag3return', 'lag4return', 'lag5return', 'lag6return',
#         'lag7return', 'step1return', 'step2return', 'step3return', 'step4return',
#         'step5return', 'step6return', 'step7return']]

# stocks['Date'] = stocks['Date'].astype('str')
# stocks['Company'] = stocks['Company'].astype('str')

# tradingdays = stocks['Date'].unique()
# tradingdays = tradingdays.tolist()

# # In[2]:

# tweets = pd.read_csv('tweet.csv')

# #thanks to Alex Kozlov/@aramacus for identifying bots
# #https://www.kaggle.com/aramacus/bot-hunting-or-how-many-tweets-were-made-by-bots

# bots = ['CBuck81TV', 'Chardenee18', 'EngkisK', 'FAJSYSTEMS', 'IncomeREIT',
#         'Javan_Andrew', 'JoshuaCEwell', 'KingRichard128', 'MarketParse',
#         'RealTraderGeek', 'RobinhoodPromo', 'Samantha_Lamb_', 'StockSaintsPrem',
#         'StrategyAcad', 'bcarpano', 'byCyril', 'impeachTHEpotus', 'mehranhydary',
#         'myforexchart', 'rival96618425', 'snowbirdie_xo', 'tweetchristo', 'uchinatravel',
#         'williaj51', 'yinyang21198  ']

# tweets = tweets[~tweets.writer.isin(bots)]

# tweets['post_date'] = pd.to_datetime(tweets['post_date'], unit = 's')

# co_tweet = pd.read_csv('company_tweet.csv')

# tweets = pd.merge(tweets, co_tweet, on = 'tweet_id')

# del co_tweet

# tweets['post_dateUTC'] = tweets['post_date'].dt.tz_localize('UTC')

# tweets['post_dateEST'] = tweets['post_dateUTC'].dt.tz_convert('US/Eastern')

# tweets['post_datedelta'] = [int(str(x)[-4]) for x in tweets['post_dateEST']]

# tweets.loc[:, 'post_date'] = pd.to_datetime(tweets['post_date']) -  pd.to_timedelta(tweets['post_datedelta'], unit='h')

# tweets = tweets.drop(['tweet_id', 'post_dateUTC', 'post_dateEST', 'post_datedelta', 'tweet_id']   , axis = 1)

# tweets['post_date'] = np.where(tweets['post_date'].dt.hour >= 16, tweets['post_date'] + pd.Timedelta(1, unit = 'day'), tweets['post_date'])

# tweets['post_date'] = tweets['post_date'].dt.date

# tweets = tweets.rename(columns = {'writer': 'Writer',
#                         'post_date': 'Date',
#                         'body': 'Tweet',
#                         'comment_num': 'Comments',
#                         'retweet_num': 'Retweet',
#                         'like_num': 'Likes',
#                         'ticker_symbol': 'Company'})

# tweets['Company'] = tweets['Company'].str.lower()
# tweets['Date'] = tweets['Date'].astype('str')
# tweets['Company'] = tweets['Company'].astype('str')

# def dayshift(date):
    
#     attempts = 0
    
#     while date not in tradingdays and attempts < 7:
#         date = datetime.fromisoformat(date) + timedelta(days = 1)
#         date = str(date.date())
#         attempts += 1
    
#     return date

# tweets['Date'] = tweets['Date'].apply(lambda x: dayshift(x))

# In[3]:

# df = pd.merge(tweets, stocks, how = 'left', on = ['Date', 'Company'])

# analyzer = SentimentIntensityAnalyzer()

# df['compound'] = [analyzer.polarity_scores(x)['compound'] for x in df['Tweet']]
# df['neg'] = [analyzer.polarity_scores(x)['neg'] for x in df['Tweet']]
# df['neu'] = [analyzer.polarity_scores(x)['neu'] for x in df['Tweet']]
# df['pos'] = [analyzer.polarity_scores(x)['pos'] for x in df['Tweet']]


# In[]:
# feather.write_dataframe(df, 'masterdf.feather')

**Sentiment Indexing and Analysis**

This part aggregates the tweet sentiments by date and company and analyzes using simple linear correlation.

Three caveats:

* It is likely linear correlation is not the optimal correlation analysis tool here. It is only a starting point. A better analysis would analyze normal and tail correlations, as a linear assumption is likely too simplistic.

* This analysis should probably be expanded to a time series approach. Autocorrelations in the tweets and stocks are likely present and not accounted for here.

* The engagement index assumes an equal weight to likes, retweets, and comments. This is probably overly simplistic, as the importance of these features may not be equal.

With that out of the way, lets look at what we have.

In [ ]:
#import packages
import pandas as pd
from matplotlib import rcParams, pyplot as plt
import seaborn as sns
import feather
from scipy. stats import pearsonr
import math

#set pic resolution
dpi = 96
plt.rcParams['figure.figsize'] = (1280/dpi, 720/dpi)

#read in data
df = pd.read_feather('../input/masterdf/masterdf.feather')


Here we will take a basic look at the composite sentiment distribution for all tweets.

It appears most score 0, or neutral, and the distribution has extreme behavior in the tails.

In [ ]:
sns.histplot(data = df.sample(frac = .2), x = 'compound', bins = 25)

Our next plot is the sotck returns. Similar to the sentiment, it is cenetered around 0, and has extreme behavior in the tails.

In [ ]:
sns.histplot(data = df.sample(frac = .2), x = 'return', bins = 25)

This next section will aggregate the tweet and engagements.

An engagement index is made by fidning the weighted average sentiment by using a tweets total engagement (i.e. retweet + like + comment).

The tweet sentiment is just the unweighted average of tweets by company and date.

Summary stats of total engagement before aggregation follows. We can tell most tweets recieve very little or no engagement.

In [ ]:
df['Engagement'] = df['Likes'] + df['Retweet'] + df['Comments']

print(df['Engagement'].describe())

Next we build the sentiment index df.

In [ ]:
#create weighted sentiment
df['WeightSent'] = df['Engagement'] * df['compound']

#aggregate by date/company
SentIndex = df.groupby(['Date', 'Company']).agg({'compound': 'mean', 
                                      'WeightSent': 'sum',
                                      'Engagement': 'sum'})
#reset index
SentIndex = SentIndex.reset_index()

#create weighted average index
SentIndex['EngIndex'] = SentIndex['WeightSent']/SentIndex['Engagement']

#get unique set of trading days and bind the aggregated sentiment to returns for each date/company
Returns = df[['Date', 'Company', 'return', 'lag1return', 'lag2return', 'lag3return', 'lag4return',
       'lag5return', 'lag6return', 'lag7return', 'step1return', 'step2return','step3return', 
       'step4return', 'step5return', 'step6return', 'step7return']]

Returns = Returns.drop_duplicates()

SentIndex = pd.merge(SentIndex, Returns, how = 'left')

SentIndex = SentIndex.drop(['WeightSent', 'Engagement'], axis = 1)

SentIndex.rename(columns={'compound':'AvgTweetSent'}, inplace=True)

Now that we have aggregated data lets look at the correlation heatmap.

In [ ]:
sns.heatmap(data = SentIndex[['AvgTweetSent', 'EngIndex', 'return', 'lag1return',
       'lag2return', 'lag3return', 'lag4return', 'lag5return', 'lag6return',
       'lag7return', 'step1return', 'step2return', 'step3return',
       'step4return', 'step5return', 'step6return', 'step7return']].corr(), cmap = 'Blues')

The plot makes it clear tweet sentiment and engagement are correlated.

It also suggeests that maybe the returns are correlated as well.

We will look at the two days before and after a tweet more closely.

In [ ]:
sns.heatmap(data = SentIndex[['AvgTweetSent', 'EngIndex', 'return', 'lag1return',
       'lag2return', 'step1return', 'step2return']].corr(), cmap = 'Blues', annot = True)

This gives us a little closer look. We can see that the return for the proceeding trading session has a positive correlation with the tweet sentiment, and less so as you move ahead through time.

What is more surprising is that the lag returns arent as strongly correlated. One would assume that if the stock is performing good/bad we would see that crop up in the tweets, possibly more so than in the next trading session since the people tweeting know what has transpired.

In [ ]:
sns.kdeplot(data = SentIndex.sample(frac = .1), x = 'AvgTweetSent', y = 'return', fill = True)

print(pearsonr(SentIndex['AvgTweetSent'], SentIndex['return']))

The above kernel density plot indicates there is likely a relationship between the average tweet sentiment and the return that day.

Our pearson r test also seems to suggest there is a statistically significant correlation between return and average tweet sentiment.



In [ ]:
sns.kdeplot(data = SentIndex.sample(frac = .1), x = 'EngIndex', y = 'return', fill = True)

print(pearsonr(SentIndex['EngIndex'], SentIndex['return']))

This kernel density plot also suggests a relationship between the engagement index and the return for that day.

The correlation also seems to be significant as well.